In [1]:
import pandas as pd
import numpy as np


In [2]:
#open dataset
df = pd.read_json('dataset.json')
df.head()

,stations,full_timetable
0,"{'Златоуст (1)': ['0', '38', '38', '25', '29',...","{'853': {'route': ['1', '3', '2'], 'free_carri..."
1,"{'Златоуст (1)': ['0', '28', '24', '28', '8', ...","{'896': {'route': ['1', '3', '2'], 'free_carri..."
2,"{'Златоуст (1)': ['0', '25', '15', '6', '15', ...","{'309': {'route': ['1', '3', '2'], 'free_carri..."
3,"{'Златоуст (1)': ['0', '32', '30', '13', '4', ...","{'800': {'route': ['2', '3', '6', '5'], 'free_..."
4,"{'Златоуст (1)': ['0', '10', '38', '16', '1', ...","{'893': {'route': ['1', '3', '6', '5', '7'], '..."


In [3]:
#check the shape of the datafile
df.shape

(100000, 2)

In [4]:
#смотрим пример данных из первой колонки 
df.iloc[0,0]

{'Златоуст (1)': ['0', '38', '38', '25', '29', '7', '10'],
 'Кыштым (2)': ['26', '0', '7', '34', '20', '27', '35'],
 'Миасс (3)': ['15', '5', '0', '27', '16', '31', '24'],
 'Муслюмово (4)': ['12', '39', '1', '0', '2', '14', '20'],
 'Челябинск (5)': ['38', '38', '1', '28', '0', '33', '14'],
 'Полетаево (6)': ['21', '27', '24', '9', '1', '0', '3'],
 'Еманжелинск (7)': ['9', '3', '23', '25', '32', '37', '0']}

In [5]:
#смотрим пример данных из второй колонки
df.iloc[0,1]

{'853': {'route': ['1', '3', '2'],
  'free_carriage': ['21', '21'],
  'timetable': ['02:00 - 02:38', '04:48 - 06:11', '13:35 - 13:58']},
 '801': {'route': ['1', '3', '2'],
  'free_carriage': ['34', '10'],
  'timetable': ['03:24 - 04:22', '06:32 - 07:06', '14:30 - 15:39']},
 '563': {'route': ['1', '3', '6', '5', '4'],
  'free_carriage': ['33', '20', '35', '3'],
  'timetable': ['03:43 - 04:26',
   '06:36 - 08:04',
   '11:42 - 11:59',
   '13:21 - 14:32',
   '16:42 - 17:34']},
 '182': {'route': ['1', '3', '6', '5', '7'],
  'free_carriage': ['15', '18', '13', '36'],
  'timetable': ['02:44 - 03:14',
   '05:24 - 05:56',
   '09:34 - 11:00',
   '12:22 - 13:05',
   '15:43 - 16:34']},
 '643': {'route': ['1', '3', '6', '5', '7'],
  'free_carriage': ['18', '1', '13', '21'],
  'timetable': ['03:23 - 03:48',
   '05:58 - 07:21',
   '10:59 - 11:20',
   '12:42 - 13:03',
   '15:41 - 16:15']},
 '743': {'route': ['2', '3', '1'],
  'free_carriage': ['8', '33'],
  'timetable': ['01:00 - 02:01', '09:25 - 10:3

In [6]:
#create initial matrix for stations with RC
def station_matrix(k, df):
    df = df
    stations = {}
    for i in range(1,8):
        m1 = df.iloc[k,0][list(df.iloc[k,0].keys())[i-1]]
        res = [int(j) for j in m1]
        stations[i]=res
    stations_df = pd.DataFrame.from_dict(stations)
    stations_df.index =[1,2,3,4,5,6,7]
    return stations_df

In [7]:
#create initial empty matrix for stations with potential routes
def station_routes():
    stations = {}
    for i in range(1,8):
        stations[i]=[[],[],[],[],[],[],[]]
    stations_df = pd.DataFrame.from_dict(stations)
    stations_df.index =[1,2,3,4,5,6,7]
    return stations_df

In [8]:
#get the list of routes from dict:
def get_routes(situation):
    routes_list = []
    for key in situation.keys():
        routes_list.append(situation[key]['route'])
    #print(routes_list)
    return routes_list

In [9]:
#create one iteration over the situation (trains remove cars) with max long haul dist
def iteration(df_trains, df_routes, rc_matrix, output):
    df_trains = df_trains
    df_routes = df_routes
    output=output
    counter = len(output)
    df_trains_tmp = df_trains.copy()
    for i in range(len(df_routes)):
        start = int(df_routes.route[i][0])
        end = int(df_routes.route[i][-1])
        #print(start,end)
        for j in range(len(df_trains)):
            #print(df_trains.loc[j,'route'][0], df_trains.loc[j,'route'][-1])
            if (df_trains.loc[j,'route'][0]==str(start))&(df_trains_tmp.loc[j,'route'][-1]==str(end)):
                
                #move cars from starting station to end_station

                #print('long_haul ', rc_matrix.loc[end,start],df_trains_tmp.loc[j,'long_slot'])
                long_haul = min(rc_matrix.loc[end,start],df_trains_tmp.loc[j,'long_slot'])
                #print('long_haul ',long_haul)
                rc_matrix.loc[end,start]-=long_haul
                rc_matrix.loc[end,end]+=long_haul
                for k in range(len(df_trains_tmp.loc[j,'route'])):
                    #print(k)
                    try:
                        df_trains_tmp.loc[j,'slot_'+str(k+1)] -= long_haul

                    except:
                        p=5 
                if long_haul>0:
                    output.loc[counter,'Станция отправления'] = str(start)
                    output.loc[counter,'Станция конечная'] =str(end)
                    output.loc[counter,'Станция промежуточная'] = ''
                    output.loc[counter,'Число вагонов'] = long_haul
                    output.loc[counter,'№ маршрута'] = df_trains_tmp.loc[j,'route No']
                    output.loc[counter,'Время прибытия'] = df_trains_tmp.loc[j,'start1']
                    output.loc[counter,'Время отправки']= df_trains_tmp.loc[j,'end1'] 
                    counter+=1    
                df_trains_tmp.loc[j,'long_slot'] -=long_haul
    return df_trains_tmp, rc_matrix, output  

In [10]:
#fill in the matrix of routes for all stations with sorted list of routes
def fill_route_matrix(df, n):
    df=df
    stationroutes = station_routes()
    for i in range(1,8):
        for j in range(1,8):
            if i!=j:
                stationroutes.iloc[i-1,j-1] = find_route(i,j, find_unique_routes(df, n))
    return stationroutes

In [11]:
#finds unique routes for the specific period
def find_unique_routes(df, n):
    df=df
    routes=[]
    situation=df.iloc[n,1]
    routes+=get_routes(situation)
    
    unique = set()
    for my_list in routes:
        unique.add(tuple(my_list))
    unique_routes = list(unique)
    routes_sorted = sorted(unique_routes)
    return routes_sorted

In [12]:
#create sorted by length list of routes 
def find_route(i,j, route_list):
    routes = []
    #print(route_list)
    for route in route_list:
        #print(route)
        try:
            start = route.index(str(i))
            end = route.index(str(j))
            #print(i,j)
        except:
            continue
        r=route[start:end+1]
        if not(r):
            continue
        else:
            routes.append(route[start:end+1])  
    routes = list(set(routes))
    
    routes = [list(item) for item in routes]
    routes.sort(key = len)

    return routes
    

In [13]:
def df_route(route_matrix):
    df_routes = pd.DataFrame(columns = ['route', 'min_len', 'max_len'])
    for i in range(7):
        for j in range(7):
            if i!=j:

                name = str(j+1)+'_'+str(i+1)
                #print(name)
                adict = {}
                length = [len(x) for x in route_matrix.iloc[j,i]]
                if length!=[]:
                    min_len = min(length)
                    max_len = max(length)
                    adict['route'] = [name]
                    adict['min_len'] = [min_len]
                    adict['max_len'] = [max_len]
                    ddf = pd.DataFrame.from_dict(adict)
                    df_routes = pd.concat([df_routes,ddf], axis=0, ignore_index=True)
                df_routes.sort_values('min_len', inplace=True, ascending=False)
    return df_routes


In [14]:
#create a matrix of trains with all the parameters
def trains(n, df):
    df=df
    df_trains = pd.DataFrame(columns = ['route No','route', 'start1', 'end1', 'slot_1','start2', 'end2', 'slot_2', 'start3', 'end3', 'slot_3', 'start4', 'end4', 'slot_4', 'start5', 'end5', 'slot_5', 'start6', 'end6', 'slot_6', 'long_slot'])
    for i in range(len(df.iloc[n,1])):
        df_trains.loc[i,'route No'] = list(df.iloc[n,1].keys())[i]
        df_trains.loc[i,'route'] = df.iloc[n,1][list(df.iloc[n,1].keys())[i]]['route']
        route_len = len(df_trains.loc[i,'route'])
        for j in range(route_len):
            df_trains.loc[i,'start'+str(j+1)] = df.iloc[n,1][list(df.iloc[n,1].keys())[i]]['timetable'][j][:5]
            df_trains.loc[i,'end'+str(j+1)] = df.iloc[n,1][list(df.iloc[n,1].keys())[i]]['timetable'][j][7:]
            try:
                df_trains.loc[i,'slot_'+str(j+1)] = int(df.iloc[n,1][list(df.iloc[n,1].keys())[i]]['free_carriage'][j])
            except:
                k=5
            tmp_list  = [int(x) for x in df.iloc[n,1][list(df.iloc[n,1].keys())[i]]['free_carriage']]
            df_trains.loc[i,'long_slot'] = min(tmp_list)
    df_trains.sort_values('end1', inplace=True)
    df_trains.reset_index(drop=True,inplace=True)
    return df_trains


In [15]:
#function to update the list of trains/slots after any iteration
def update_trains(df_trains_tmp):
    df_trains_tmp=df_trains_tmp
    df_trains_update = pd.DataFrame(columns = ['route No','route', 'start1', 'end1', 'slot_1','start2', 'end2', 'slot_2', 'start3', 'end3', 'slot_3', 'start4', 'end4', 'slot_4', 'start5', 'end5', 'slot_5', 'start6', 'end6', 'slot_6', 'long_slot'])
    #df_trains_update
    update_count=0#check if minterrupted with 0 slot or new train


    for i in range(len(df_trains_tmp)):#len(df_trains_tmp)
        route_count=0 #count length of route
        line_check=1 #check if minterrupted with 0
        long_slot=0
        route=[]
        train_no=df_trains_tmp.iloc[i,:]['route No']

        for k in range(len(df_trains_tmp.iloc[i,:].route)):
            #add train no, station info, slots
            #route=[df_trains_tmp.iloc[i,:]['route'][k]]

            if (k>0)&(route_count==0): route=[]

            if df_trains_tmp.iloc[i,:]['slot_'+str(k+1)]>0:

                route_count+=1
                route+=[df_trains_tmp.iloc[i,:]['route'][k]]
                #print('rc ',route)
                slot = df_trains_tmp.iloc[i,:]['slot_'+str(k+1)]   
                start = df_trains_tmp.iloc[i,:]['start'+str(k+1)]
                end  = df_trains_tmp.iloc[i,:]['end'+str(k+1)]
                #long_slot=min(slot, long_slot)
                #write this
                df_trains_update.loc[update_count,'route No'] = train_no
                df_trains_update.loc[update_count,'route'] = route
                df_trains_update.loc[update_count,'slot_'+str(route_count)] = slot
                df_trains_update.loc[update_count,'start'+str(route_count)] = start
                df_trains_update.loc[update_count,'end'+str(route_count)] = end
                df_trains_update.loc[update_count,'long_slot'] = long_slot

            else:
                if route_count > 0:

                    start = df_trains_tmp.iloc[i,:]['start'+str(k+1)]
                    end  = df_trains_tmp.iloc[i,:]['end'+str(k+1)]
                    #long_slot=min(slot, long_slot)
                    df_trains_update.loc[update_count,'start'+str(route_count+1)] = start
                    df_trains_update.loc[update_count,'end'+str(route_count+1)] = end
                    route+=[df_trains_tmp.iloc[i,:]['route'][k]]
                    update_count+=1
                    route_count=0
                else:

                    continue
    df_trains_update.sort_values('end1', inplace=True)
    df_trains_update.reset_index(drop=True,inplace=True)
    df_trains_update['long_slot'] = [min(row[0],row[1],row[2],row[3],row[4], row[5]) for row in zip(df_trains_update['slot_1'],df_trains_update['slot_2'],df_trains_update['slot_3'],df_trains_update['slot_4'],df_trains_update['slot_5'],df_trains_update['slot_6'])]

    return df_trains_update
    
     

In [26]:
%%time
#Осноная часть для запуска (данные берутся из предложенного сета, n-номер ситуации)

n=40
rc_matrix = station_matrix(n, df)
res0 = pd.Series(np.diag(rc_matrix), index=[rc_matrix.index, rc_matrix.columns]).sum()
print('Исходные данные по станциям:')
print(rc_matrix)
print('total cars initially: ', rc_matrix.sum().sum())
#создание драфта выходного документа
output = {'Станция отправления': [], 'Станция конечная': [], 'Станция промежуточная': [],'Число вагонов': [], '№ маршрута': [],'Время прибытия': [], 'Время отправки': []} 
output = pd.DataFrame.from_dict(output)
#создание графика подачи составов в формате, удобном для расчетов
df_trains = trains(n, df)
#find the unique routes for trains between stations (graph for current case trains routes)
route_list=find_unique_routes(df, n)
#create a matrix of possible routes for pairs of stations 
route_matrix = fill_route_matrix(df, n)
df_routes = df_route(route_matrix)
#Let's start the first iteration and get the iteration resulting matrix of RC left
df_trains_tmp, rc_matrix_updated, output = iteration(df_trains, df_routes, rc_matrix, output)
#rc_matrix_updated - матрица исходная
print('Матрица графов проходящих маршрутов между станциями:')
print(route_matrix)
res0 = pd.Series(np.diag(rc_matrix), index=[rc_matrix.index, rc_matrix.columns]).sum()
res = pd.Series(np.diag(rc_matrix_updated), index=[rc_matrix_updated.index, rc_matrix_updated.columns]).sum()
#Вторая и последующие итерации до момента пркращения улучшения ситуации
moved = res-res0
while moved:
    
    #update the train matrix to reflect used slots
    df_trains_tmp=update_trains(df_trains_tmp) 
    df_trains_tmp, rc_matrix_updated, output=iteration(df_trains_tmp, df_routes, rc_matrix_updated, output)
    res = pd.Series(np.diag(rc_matrix_updated), index=[rc_matrix_updated.index, rc_matrix_updated.columns]).sum()
    moved = res-res0
    print('RC moved: ', moved)
    res0=res
print('total cars left ',rc_matrix_updated.sum().sum()-pd.Series(np.diag(rc_matrix_updated), index=[rc_matrix_updated.index, rc_matrix_updated.columns]).sum())
print('Состояние на конец смены:')
rc_matrix_updated_0 = rc_matrix_updated
for i in range(7):
    rc_matrix_updated_0.iloc[i,i]=''
print(rc_matrix_updated_0)
#вывод файла с рекомендациями для диспетчеров (сортоирован по станциям)
output.sort_values('Станция отправления', inplace=True)
output.to_csv('output.csv', index=False)

Исходные данные по станциям:
    1   2   3   4   5   6   7
1   0  28   8  19  33  14  32
2  24   0  13  17   2  36  23
3  15  31   0   2  22  10  26
4  18  11   1   0  34  34   8
5  37  29   1   6   0   6   4
6  33  29   2   3  31   0   5
7  36  31  17   3   0  23   0
total cars initially:  757
Матрица графов проходящих маршрутов между станциями:
    1               2            3                          4  \
1  []              []     [[1, 3]]                         []   
2  []              []     [[2, 3]]                         []   
3  []        [[3, 2]]           []                         []   
4  []              []           []                         []   
5  []  [[5, 6, 3, 2]]  [[5, 6, 3]]                   [[5, 4]]   
6  []     [[6, 3, 2]]     [[6, 3]]                [[6, 5, 4]]   
7  []              []           []  [[7, 5, 4], [7, 6, 5, 4]]   

                     5            6                    7  
1       [[1, 3, 6, 5]]  [[1, 3, 6]]                   []  
2           

In [ ]:
df_routes